<a href="https://colab.research.google.com/github/Arkaprabha1012/pythonopencv/blob/main/ResnetOnCovidpneunormal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arkaprabha1012","key":"f07b1a3ffd2b86f82a0788e5e9dc8dfe"}'}

In [3]:
! mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                       title                                         size  lastUpdated          downloadCount  
--------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  
michau96/restaurant-business-rankings-2020                Restaurant Business Rankings 2020             16KB  2021-01-30 14:20:45           4917  
gpreda/reddit-wallstreetsbets-posts                       Reddit WallStreetBets Posts                    9MB  2021-03-02 07:25:19           2142  
yorkehead/stock-market-subreddits                         Stock Market Subreddits                        1MB  2021-01-29 13:53:50            421  
aagghh/crypto-telegram-groups                             Crypto telegram groups                       297MB  2021-02-02 09:58:25            253  
timoboz/superbowl-history-1967-2020                       Superbowl History 1967 - 2020                  2KB  2020-02-

In [7]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

100% 1.13G/1.14G [00:08<00:00, 135MB/s]
100% 1.14G/1.14G [00:08<00:00, 144MB/s]


In [8]:
!unzip covid19-radiography-database.zip

Archive:  covid19-radiography-database.zip
  inflating: COVID-19 Radiography Database/COVID.metadata.xlsx  
  inflating: COVID-19 Radiography Database/COVID/COVID (1).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (10).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (100).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1000).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1001).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1002).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1003).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1004).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1005).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1006).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1007).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1008).png  
  inflating: COVID-19 Radiography Database/COVID/COVID (1009).png  
  inflating: C

In [9]:
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions,ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
pip install split-folders tqdm

In [13]:
#rm "/content/COVID-19RadiographyDatabase/.xlsx"
!rmdir "/content/COVID-19RadiographyDatabase/.ipynb_checkpoints"

In [14]:
import splitfolders  # or import split_folders
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.

splitfolders.ratio("/content/COVID-19RadiographyDatabase", output="Output", seed=1337, ratio=(.85, .15), group_prefix=None) # default values
# Split val/test with a fixed number of items e.g. 100 for each set.
# To only split into training and validation set, use a single number to `fixed`, i.e., `10`.
splitfolders.fixed("/content/COVID-19RadiographyDatabase", output="Output", seed=1337, fixed=(70,70), oversample=False, group_prefix=None) # default values


Copying files: 0 files [00:00, ? files/s]
Copying files: 152 files [00:00, 1515.20 files/s]
Copying files: 347 files [00:00, 1622.91 files/s]
Copying files: 478 files [00:00, 1507.73 files/s]
Copying files: 580 files [00:00, 1237.56 files/s]
Copying files: 694 files [00:00, 1204.59 files/s]
Copying files: 813 files [00:00, 1199.87 files/s]
Copying files: 947 files [00:00, 1237.44 files/s]
Copying files: 1077 files [00:00, 1189.81 files/s]
Copying files: 1191 files [00:01, 850.71 files/s] 
Copying files: 1286 files [00:01, 766.43 files/s]
Copying files: 1469 files [00:01, 912.70 files/s]
Copying files: 1704 files [00:01, 1116.47 files/s]
Copying files: 1971 files [00:01, 1352.26 files/s]
Copying files: 2312 files [00:01, 1650.97 files/s]
Copying files: 2584 files [00:01, 1871.10 files/s]
Copying files: 2827 files [00:02, 1172.08 files/s]
Copying files: 3016 files [00:02, 795.94 files/s] 
Copying files: 3162 files [00:02, 757.96 files/s]
Copying files: 3285 files [00:03, 641.91 files/s]

In [15]:
img_height,img_width=(224,224)
batch_size=128
train_data_dir="/content/Output/train"
valid_data_dir="/content/Output/val"
!rmdir "/content/Output/train/.ipynb_checkpoints/"
!rmdir "/content/Output/test/.ipynb_checkpoints/"
#test_data_dir="/content/Output/test"

rmdir: failed to remove '/content/Output/train/.ipynb_checkpoints/': No such file or directory
rmdir: failed to remove '/content/Output/test/.ipynb_checkpoints/': No such file or directory


In [16]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 validation_split=0.4)
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  subset='training')
valid_generator=train_datagen.flow_from_directory(valid_data_dir,
                                                  target_size=(img_height,img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  subset='validation')

print(valid_generator.class_indices)
print(train_generator.class_indices)


Found 2080 images belonging to 3 classes.
Found 232 images belonging to 3 classes.
{'COVID': 0, 'NORMAL': 1, 'ViralPneumonia': 2}
{'COVID': 0, 'NORMAL': 1, 'ViralPneumonia': 2}


In [17]:
base_model=ResNet50(include_top=False,
                    weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(train_generator.num_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=predictions)
for layer in base_model.layers:
  layer.trainable=False
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

94773248/94765736 [==============================] - 1s 0us/step


In [18]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint=ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')
callbacks_list=[checkpoint]

In [ ]:
'''r = model.fit_generator(
  train_generator,
  epochs=10,
  steps_per_epoch=len(valid_generator),
  validation_data=valid_generator,
  validation_steps=len(valid_generator),
  callbacks=callbacks_list
)'''
r = model.fit_generator(
  train_generator,
  epochs=100,
  steps_per_epoch=len(valid_generator),
  validation_data=valid_generator,
  validation_steps=2000//(batch_size*10),
  callbacks=callbacks_list
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
2/2 [==============================] - 45s 9s/step - loss: 1.5894 - accuracy: 0.4141 - val_loss: 1.7325 - val_accuracy: 0.6406

Epoch 00001: val_accuracy improved from -inf to 0.64062, saving model to weights-improvement-01-0.64.hdf5
Epoch 2/100
2/2 [==============================] - 12s 9s/step - loss: 1.3152 - accuracy: 0.6615 - val_loss: 0.3191 - val_accuracy: 0.8125

Epoch 00002: val_accuracy improved from 0.64062 to 0.81250, saving model to weights-improvement-02-0.81.hdf5
Epoch 3/100
2/2 [==============================] - 12s 9s/step - loss: 0.3363 - accuracy: 0.8542 - val_loss: 0.7085 - val_accuracy: 0.7188

Epoch 00003: val_accuracy did not improve from 0.81250
Epoch 4/100
2/2 [==============================] - 11s 9s/step - loss: 0.4119 - accuracy: 0.8776 - val_loss: 0.2922 - val_accuracy: 0.9219

Epoch 00004: val_accuracy improved from 0.81250 to 0.92188, saving model to weights-improvement-04-0.92.hdf5
Epoch 5/100
2/2 [==============================] - 10s 7s/ste

In [ ]:
model.save("resenet50medical.h5")

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import keras
from tensorflow.keras import utils
# We need not to create and train the model again
# No need to train the model, its a pre defined trained model
model = load_model("/content/resenet50medical.h5")
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Testing the Normal Image
image = cv2.imread("/content/paving.jpg")
# image = cv2.imread("covid19dataset/test/covid/nejmoa2001191_f3-PA.jpeg")    # 0
image = cv2.resize(image, (128, 128))
image = np.reshape(image, [1, 128, 128, 3])
classes = model.predict(image)
result=np.argmax(classes)
if result==0:
  print("COVID")
elif result==1:
  print("NORMAL")
else:
  print("PNEUMONIA")

COVID
